# Explore Valley Wind Events

Author: Daniel Hogan

Created: January 4, 2024

**Description:** This notebook seeks to establish when high wind events occurred within the East River valley in Colorado during the SAIL and SOS field campaigns. We will begin by looking at a specific event during the final days of 2022 and then gather some information about the atmospheric characteristics by looking at vertical wind profiles, radiosondes, and reanalysis data. We will then move on to a few other case studies from the analysis period to compare some of these different wind events and how valley orientation plays a role by starting to look at a few different wind observations around the region. 

### Imports

In [1]:
import xarray as xr 
from sublimpy import utils, variables, tidy
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import os
import cufflinks as cf
from act import discovery, plotting
import json
from scripts.get_sail_data import get_sail_data
from scripts.helper_funcs import create_windrose_df
from metpy import calc, units
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()

nctoolkit is using Climate Data Operators version 2.3.0


# December 21 - December 27, 2022
## SOS Data Download

In [2]:
sos_5min_data = utils.download_sos_data(
                                        start_date='20221221',
                                        end_date='20221227',
                                        variable_names=variables.DEFAULT_VARIABLES,
                                        local_download_dir="../../01_data/raw_data/sosnoqc/",
                                        cache=True
                                    )                                  

Caching...skipping download for 20221221
Caching...skipping download for 20221222
Caching...skipping download for 20221223
Caching...skipping download for 20221224
Caching...skipping download for 20221225
Caching...skipping download for 20221226
Caching...skipping download for 20221227


## SAIL Data Download

Data was downloaded from the [ARM User Facility](https://www.arm.gov/research/campaigns/amf2021sail).
Variables downloaded include:

*Measured*

- Balloon-borne sounding system (BBSS): Vaisala-processed winds, pressure, temp, &RH (sondewnpn) released twice per day released from Gothic
- Surface meteorology at 10 m (met) located in Gothic
- Quality Controlled Eddy Correlation Flux Measurement - 30 min averaged (30qcecor) located at Kettle Ponds
- 915-MHz Radar Wind Profiler/RASS (RWP915): wind consensus data (915rwpwindcon) located in Gothic
- Doppler lidar wind value-added product (dlprofwind4news) located in Gothic
- Laser disdrometer (ld). One located in Gothic (M1) and the other located at Kettle Ponds (S2)

*Modeled*

- ECMWF: near-surface and surface variables (extra), entire coverage, 1-hr avg (ecmwfsfce)
- ECMWF: model multilevel surface fields at 4 levels, entire coverage (ecmwfsfcml)
- ECMWF: model met. and cloud variables at altitude, entire coverage, 1-hr avg (ecmwfvar)

In [3]:
# Function to load ARM credentials
def load_arm_credentials(credential_path):
    with open(credential_path, 'r') as f:
        credentials = json.load(f)
    return credentials
# Location of ARM credentials
credential_path = '/home/dlhogan/.act_config.json'
credentials = load_arm_credentials(credential_path)
# api token and username for ARM
api_username = credentials.get('username')
api_token = credentials.get('token')

In [4]:
# dictionary to store datastream names
sail_datastream_dict = {
    "radiosonde":"gucsondewnpnM1.b1",
    "met":"gucmetM1.b1",
    "eddy_covariance_kettle_ponds":"guc30qcecorS3.s1",
    "wind_profiler":"guc915rwpwindconM1.a1",
    "doppler_lidar":"gucdlprofwind4newsM1.c1",
    "laser_disdrometer_gothic":"gucldM1.b1",
    "laser_disdrometer_mt_cb":"gucldS2.b1",
    "ceilometer_10m_backscatter":"gucceil10mM1.b1",
}

In [5]:
# Set the location of the data folder where this data will be stored
event = 'dec2022'
sail_event_data_folder = f'/home/dlhogan/GitHub/Synoptic-Sublimation/01_data/raw_data/sail_{event}/'
# Set the start and end dates that we want to pull
startdate = '20221221'
enddate = '20221227'
# create empty data dictionary
sail_data_loc_dict = {}
# Iterate through the dictionary and pull the data for each datastream
for k,v in sail_datastream_dict.items():
    if (k =='radiosonde') & (len(os.listdir(os.path.join(sail_event_data_folder,"radiosonde"))) > 0):
        print("Radiosonde data donwloaded. Data files include:")
        # list file names in the radiosonde folder
        for file in os.listdir(os.path.join(sail_event_data_folder,"radiosonde")):
            print(file)
        print('-------------------')
    # Check if the file already exists
    elif (os.path.exists(f'{sail_event_data_folder}{k}_{startdate}_{enddate}.nc')): 
        print(f'{k}_{startdate}_{enddate}.nc already exists')
        print('-------------------')
        # add the filename to the dictionary which can be used if we want to load the data
        sail_data_loc_dict[k] = os.path.join(sail_event_data_folder,f'{k}_{startdate}_{enddate}.nc')
        continue
    else:
        # explicitly download radiosonde data because they are a lot easier to process and think about when in individual files
        if k == 'radiosonde':
            discovery.download_data(
                api_username,
                api_token,
                v,
                startdate=startdate,
                enddate=enddate,
                output=sail_event_data_folder+'radiosonde/'
            )
        else:
            ds = get_sail_data(api_username,
                        api_token,
                        v,
                        startdate=startdate,
                        enddate=enddate)
            ds.to_netcdf(f'{sail_event_data_folder}{k}_{startdate}_{enddate}.nc')
            sail_data_loc_dict[k] = os.path.join(sail_event_data_folder,f'{k}_{startdate}_{enddate}.nc')

Radiosonde data donwloaded. Data files include:
gucsondewnpnM1.b1.20221222.113600.cdf
gucsondewnpnM1.b1.20221223.112600.cdf
gucsondewnpnM1.b1.20221222.232700.cdf
gucsondewnpnM1.b1.20221223.233300.cdf
gucsondewnpnM1.b1.20221224.113000.cdf
gucsondewnpnM1.b1.20221224.232900.cdf
gucsondewnpnM1.b1.20221225.112600.cdf
gucsondewnpnM1.b1.20221225.232900.cdf
gucsondewnpnM1.b1.20221226.112800.cdf
gucsondewnpnM1.b1.20221226.232500.cdf
-------------------
met_20221221_20221227.nc already exists
-------------------
eddy_covariance_kettle_ponds_20221221_20221227.nc already exists
-------------------
wind_profiler_20221221_20221227.nc already exists
-------------------
doppler_lidar_20221221_20221227.nc already exists
-------------------
laser_disdrometer_gothic_20221221_20221227.nc already exists
-------------------
laser_disdrometer_mt_cb_20221221_20221227.nc already exists
-------------------
ceilometer_10m_backscatter_20221221_20221227.nc already exists
-------------------


## Synoptic Data 

Synoptic data was downloaded from their API service. Data is contained in the `raw_data/station_data` folder.  It was downloaded using the [Synoptic data downloading tool](https://download.synopticdata.com/) for stations within the East River region.

In [6]:
# synoptic file path
filepath = '../../01_data/raw_data/station_data/'
filenames = os.listdir(filepath)

def convert_synoptic_datatypes(df):
    """
    This function takes in a dataframe and converts the types of the columns to the correct types
    Inputs:
        df: pandas dataframe
    Outputs:
        df: pandas dataframe with the correct types
    """
    for col in df.columns:
        if col == 'Date_Time':
            # convert the date time column to datetime
            df.Date_Time = pd.to_datetime(df.Date_Time)
        elif col == 'Station_ID':
            df[col] = df[col].astype(str)
        else:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

# Create a function that takes in a filepath and name and returns a tidy dataframe
def load_synoptic_data(filepath, filename, tidy=False, with_units=False):
    """
    This function takes in a filepath and filename and returns a tidy dataframe and a dictionary of metadata
    Inputs:
        filepath: string of the filepath to the synoptic data
        filename: string of the filename to the synoptic data
    Outputs:
        tidy_df: pandas dataframe of the synoptic data in tidy format
        synoptic_metadata: dictionary of the metadata for the synoptic data
    """
    # for the first filename ignore the rows with # at the start and 
    df = pd.read_csv(filepath+filename, comment='#')
    # remove _set_1_ from the column names
    df.columns = df.columns.str.replace('_set_1','')
    # remove the first row and save as a units dictionary
    station_units = df.iloc[0].to_dict()

    # remove the first row
    df = df.iloc[1:]
    # convert the data types
    df = convert_synoptic_datatypes(df)

    if tidy:
        # convert this to a tidy dataframe
        tidy_df = df.melt(id_vars=['Station_ID','Date_Time'])
        # add units column with the units dictioanry
        tidy_df['units'] = tidy_df['variable'].map(station_units)
        tidy_df.Date_Time = pd.to_datetime(tidy_df.Date_Time)
        return tidy_df
    else:
        # remove keys from the dictionary that have nan values
        station_units = {k: v for k, v in station_units.items() if pd.notnull(v)}
        # replace Celcius with degC
        station_units = {k: v.replace('Celsius','degC').replace('Degrees','degree').replace('Millimeters','millimeter') for k, v in station_units.items()}  
        # add units to dataframe
        if with_units:
            df = units.pandas_dataframe_to_unit_arrays(df, station_units)
            return df
        else:
            return df

# save the comments as a dictionary to record metadata for later use
synoptic_metadata = {}
for filename in filenames:
    with open(filepath+filename) as f:
        id = filename.split('.')[0]
        synoptic_metadata[id] = {}
        for i,line in enumerate(f):
            if i < 4:
                continue
            elif line.startswith('#'):
                # save everything before the colon as the key and the rest as the value
                key, value = line.split(':', 1)
                # remove the # and the whitespace
                key = key[1:].strip()
                value = value.strip()
                # save the key value pair to the dictionary
                synoptic_metadata[id][key] = value
            else:
                break
            # close the file
    f.close()

## Load in the data of interest

In [15]:
# Load the met data from the sail dictionary into an xarray dataset
# SAIL Data
met_ds = xr.open_dataset(sail_data_loc_dict['met'])
ecor_ds = xr.open_dataset(sail_data_loc_dict['eddy_covariance_kettle_ponds'])
# SOS data
sos_ds = xr.open_mfdataset("../../01_data/raw_data/sosnoqc/*.nc")
sos_tidy_df = tidy.get_tidy_dataset(sos_ds, variables.DEFAULT_VARIABLES)

## Plot Kettle Pond wind speeds, blowing snow flux, and pressure

In [8]:
snow_flux_1m = sos_tidy_df.query("measurement=='snow flux'")
fig = px.line(snow_flux_1m,
        x='time',
        y='value',
        color='height')
fig.update_layout(
    title="Blowing Snow Flux",
    xaxis_title="Time",
    yaxis_title="Snow Flux (kg/m^2/s)",
    legend_title="Height above ground (m)",
    width= 800,
    height=400,
)

In [9]:
print("SOS measurements")
print("----------------")
for measure in sos_tidy_df.measurement.unique():
    print(measure)

SOS measurements
----------------
virtual temperature
wind speed
wind direction
u
v
w
u_u_
v_v_
w_w_
u_w_
v_w_
u_tc_
v_tc_
u_h2o_
v_h2o_
w_tc_
w_h2o_
temperature
RH
pressure
snow flux
Vtherm
Vpile
IDir
snow temperature
None
shortwave radiation incoming


Calculate SOS sublimation rate. This needs to be done differntly since the measured quantity is the perturbation of vertical wind speed and the perturbation of the density of water vapor. So units are naturally in $g \text{ }m^{-2}\text{ } s^{-1}$. SAIL computes theirs at $W \text{ } m^{-2}$, so in that case we need to include the latent heat of sublimation.

In [12]:
# Latent heat of vaporization and fusion
Lv = 2.5e6 # J/kg at 0 deg C
Lf = 3.34e5 # J/kg at 0 deg C
# Latent heat of sublimation
Ls = Lv + Lf
# density of water 
rho_w = 1000 # kg/m^3
# grams to kg conversion
g2kg = 1/1000 # kg/g
def latent_heat_sublimation(T):
    """
    This function takes in a temperature in degrees C and returns the latent heat of sublimation in J/kg
    from M. K. Yau (1989). A Short Course in Cloud Physics (3rd ed.). Pergamon Press. p. 16. ISBN 0-7506-3215-1.
    """
    if T <= 0:
        Ls = (2834.1 -0.29*T - (0.004*T**2))*1000
    else:
        Ls = (2500.8 - 2.36*T + (0.0016*T**2) - (0.00006*T**3))*1000
    return Ls

# calculate the sublimation rate
sos_sublimation_5min_avg = sos_tidy_df.query("measurement=='w_h2o_' and tower=='c'")
sos_sublimation_5min_avg.value = sos_sublimation_5min_avg.value * 1/rho_w * 1000 * g2kg # convert from g/m^2/s to mm/s
# set the time as the index
sos_sublimation_5min_avg = sos_sublimation_5min_avg.set_index('time')

# calculate the hourly total sublimation rate from the 5 min average on the value column
# multiply the value column by the number of seconds in an hour to get the hourly total in mm/hr
sos_sublimation_hourly = sos_sublimation_5min_avg.groupby(['height']).resample('1H').agg({'value':'mean'})*3600
# reset the inded and sort by time
sos_sublimation_hourly = sos_sublimation_hourly.reset_index().sort_values('time')

In [13]:
# create a color dictionary for the unique height values in the sos_tidy_df
color_values = sorted(list(sos_tidy_df.height.dropna().unique()))
n_colors = len(color_values)                      
color_scale = px.colors.sample_colorscale("viridis", [n/(n_colors -1) for n in range(n_colors)])
color_dict = dict(zip(color_values,color_scale))

In [12]:
# now let's do the same with facets and plot our wind speed as well
snow_flux = sos_tidy_df.query("measurement=='snow flux'")
wind_speed = sos_tidy_df.query("measurement=='wind speed'")
temperature = sos_tidy_df.query("measurement=='temperature' and tower=='c' and height==2")
pressure = sos_tidy_df.query("measurement=='pressure'")

# create a figure with 3 facets
fig = make_subplots(5,1, 
                    subplot_titles=("Blowing Snow Flux", "Wind Speed", "Sublimation", "Pressure", "Temperature"),
                    shared_xaxes=True, 
                    vertical_spacing=0.05)

# plot the blowing snow flux at the two levels as above
for height in snow_flux.height.unique():
    fig.add_trace(go.Scatter(
        x=snow_flux.query(f"height=={height}")['time'], 
        y=snow_flux.query(f"height=={height}")['value'],
        name=f"{height} m",
        marker_color=color_dict[height]
    ),
    row=1, col=1
    )
# plot the wind speed at the central tower at all heights
wind_speed_central = wind_speed.query("tower=='c'")
# round values to the nearest 10th
wind_speed_central.value = wind_speed_central.value.round(1)
for height in wind_speed_central.height.unique():
    fig.add_trace(go.Scatter(
        x=wind_speed_central.query(f"height=={height}")['time'], 
        y=wind_speed_central.query(f"height=={height}")['value'],
        name=f"{height} m",
        marker_color=color_dict[height]
    ),
    row=2, col=1,
    )
# plot the sublimation rate
for height in sos_sublimation_hourly.height.unique():
    fig.add_trace(go.Scatter(
        x=sos_sublimation_hourly.query(f"height=={height}")['time'], 
        y=sos_sublimation_hourly.query(f"height=={height}")['value'],
        name=f"{height} m",
        marker_color=color_dict[height]
    ),
    row=3, col=1,
    )
# plot the pressure at the central tower
pressure_central_10m = pressure.query("tower=='c' and height==10")
fig.add_trace(go.Scatter(
    x=pressure_central_10m['time'], 
    y=pressure_central_10m['value'],
    showlegend=False,
    marker_color=color_dict[10.0],
    name="10m Pressure"
),
row=4, col=1,
)
# plot the temperature at the central tower at 2m
fig.add_trace(go.Scatter(
    x=temperature['time'], 
    y=temperature['value'],
    showlegend=False,
    marker_color=color_dict[2.0],
    name="2m Temperature"
),
row=5, col=1,
)

# update the layout
fig.update_layout(
    title="Kettle Ponds Wind Event 2022-12-21 to 2022-12-27",
    xaxis3=dict(
        title="Time (UTC)"
    ),
    yaxis1=dict(
        title="Snow Flux<br>(g/m^2/s)"
    ),
    yaxis2=dict(
        title="Wind Speed<br>(m/s)"
    ),
    yaxis3=dict(
        title="Sublimation<br>(mm/hr)"
    ),
    yaxis4=dict(
        title="Pressure<br>(hPa)"
    ),
    yaxis5=dict(
        title="Temperature<br>(C)"
    ),
    legend_title="Measurement height\nabove ground (m)",
    hovermode="x unified",
    width= 800,
    height=800,
)
# update the legend to only show one entry for each height
# https://stackoverflow.com/questions/26939121/how-to-avoid-duplicate-legend-labels-in-plotly-or-pass-custom-legend-labels
names = set()
fig.for_each_trace(
    lambda trace:
        trace.update(showlegend=False)
        if (trace.name in names) else names.add(trace.name))

Now let's make a plot of wind direction on Dec 22nd (UTC time)

In [95]:
# gather wind speed and direction data from SOS at 10 meters on the UW tower
wind_variables = ['wind speed','wind direction']
wind_dir_10m_uw = sos_tidy_df.query("measurement in @wind_variables and tower=='uw' and height==10")
# filter to times on the 22nd of december
wind_dir_10m_uw = wind_dir_10m_uw.query("time.dt.day==22")
wind_dir = wind_dir_10m_uw[wind_dir_10m_uw['measurement']=='wind direction'][['value', 'time']].rename(columns={'value':'wind direction'}).reset_index(drop=True)
wind_speed = wind_dir_10m_uw[wind_dir_10m_uw['measurement']=='wind speed'][['value']].rename(columns={'value':'wind speed'}).reset_index(drop=True)
# combine wind speed and direction into a single dataframe
wind_dir_10m_uw = pd.concat([wind_dir, wind_speed], axis=1)
# generate a wind rose dataframe
windrose_10m_uw = create_windrose_df(wind_dir_10m_uw, 'wind direction', 'wind speed')

# create a wind rose plot using bar_polar
fig = px.bar_polar(windrose_10m_uw, 
                   r="frequency", 
                   theta="direction", 
                   color="speed", template="plotly_dark",
                   color_discrete_sequence= px.colors.sequential.Plasma_r,
                   width= 800,
                   height=500,
                   barnorm='percent',
                  )
# update the radial axis to show percentages
fig.update_layout(
    title="Wind Rose at 10 m on UW Tower on 2022-12-22",
    font_size=16,
    polar_radialaxis=dict(
         ticksuffix='%',
         tickfont_size=14,
         showline=False,
         showticklabels=True,
         showgrid=True,
         angle=45,
         range=[0, 80]
      ),
)

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.


Now we will plot the SAIL data with met data in Gothic and ECOR data at Kettle Ponds

In [15]:
# Meteorological data from SAIL met
met_vars = ['atmos_pressure', 'wspd_arith_mean','wdir_vec_mean','temp_mean']
qc_met_vars = ['qc_'+var for var in met_vars]
# get the variables from the met data
met_df = met_ds[met_vars+qc_met_vars].to_dataframe().reset_index()
# print the frequency of the data in minutes
print('Frequency of data in minutes: {}'.format((met_df.time.diff().dt.total_seconds().mean()/60)))
print('Pre-QC data length: {}'.format(len(met_df)))
# drop all values when any qc column is not 0
met_df = met_df[met_df[qc_met_vars].eq(0).all(1)]
print('Post-QC data length: {}'.format(len(met_df)))
# drop the qc columns
met_df.drop(qc_met_vars, axis=1, inplace=True)
# set time as the index
met_df = met_df.set_index('time')
# convert pressure from kPa to hPa
met_df.atmos_pressure = met_df.atmos_pressure*10
# create an 5 minute average dataset but cacluate the median for the wdir_vec_mean column
met_df_5min = met_df.resample('5min').agg({'atmos_pressure':'mean',
                                       'wspd_arith_mean':'mean',
                                       'wdir_vec_mean':'median',
                                       'temp_mean':'mean'}).reset_index()

Frequency of data in minutes: 1.0
Pre-QC data length: 10080
Post-QC data length: 10080


In [16]:
# ECOR data from SAIL ecor
ecor_vars = ['latent_heat_flux']
qc_ecor_vars = ['qc_'+var for var in ecor_vars]
# get the variables from the met data
ecor_df = ecor_ds[ecor_vars+qc_ecor_vars].to_dataframe().reset_index()
# print the frequency of the data in minutes
print('Frequency of data in minutes: {}'.format((ecor_df.time.diff().dt.total_seconds().mean()/60)))
print('Pre-QC data length: {}'.format(len(ecor_df)))
# drop all values when any qc column is not 0
ecor_df = ecor_df[ecor_df[qc_ecor_vars].eq(0).all(1)]
print('Post-QC data length: {}'.format(len(ecor_df)))
# drop the qc columns
ecor_df.drop(qc_ecor_vars, axis=1, inplace=True)
# set time as the index
ecor_df = ecor_df.set_index('time')
# resample to 1H
ecor_df_1H = ecor_df.resample('1H').agg({'latent_heat_flux':'mean'}).reset_index()
# add a sublimation column by dividing the latent heat flux by the latent heat of sublimation and converting from W/m^2 to mm/hr
ecor_df_1H['sublimation'] = ecor_df_1H['latent_heat_flux']/Ls * 3600

Frequency of data in minutes: 30.0
Pre-QC data length: 336
Post-QC data length: 270


In [17]:
fig = make_subplots(4,1,
                    subplot_titles=("Wind Speed", "Sublimation  @ Kettle Ponds","Pressure", "Temperature"),
                    shared_xaxes=True, 
                    vertical_spacing=0.05)

# plot the met_ds_5min wind speed
fig.add_trace(go.Scatter(
    x=met_df_5min['time'], 
    y=met_df_5min['wspd_arith_mean'],
    showlegend=False,
    marker_color='red',
    name='SAIL'
),
row=1, col=1,
)
# plot the sos wind speed at 10m on the UW tower

fig.add_trace(go.Scatter(
    x=wind_speed_central[wind_speed_central['height']==10]['time'], 
    y=wind_speed_central[wind_speed_central['height']==10]['value'],
    showlegend=False,
    marker_color='purple',
    name='SOS'
),
row=1, col=1,
)
# plot the ecor_ds_1H sublimation
fig.add_trace(go.Scatter(
    x=ecor_df_1H['time'], 
    y=ecor_df_1H['sublimation'],
    showlegend=False,
    marker_color='red',
    name='SAIL ECOR'
),
row=2, col=1,
)
# plot sos sublimation on top
fig.add_trace(go.Scatter(
    x=sos_sublimation_hourly[sos_sublimation_hourly['height']==5]['time'], 
    y=sos_sublimation_hourly[sos_sublimation_hourly['height']==5]['value'],
    showlegend=False,
    marker_color='purple',
    name='SOS EC'
),
row=2, col=1,
)
# plot the met_ds_5min pressure
fig.add_trace(go.Scatter(
    x=met_df_5min['time'], 
    y=met_df_5min['atmos_pressure'],
    showlegend=False,
    marker_color='green',
    name='Pressure'
),
row=3, col=1,
)
# plot the met_ds_5min temperature
fig.add_trace(go.Scatter(
    x=met_df_5min['time'], 
    y=met_df_5min['temp_mean'],
    showlegend=False,
    marker_color='orange',
    name='2m Temperature',
),
row=4, col=1,
)
# add horizontal line at 0 for temperature
fig.add_shape(
        type="line",
        x0=met_df_5min['time'].min(),
        y0=0,
        x1=met_df_5min['time'].max(),
        y1=0,
        line=dict(
            color="black",
            width=2,
            dash="dashdot",
        ),
    row=4, col=1,

)

# update the layout
fig.update_layout(
    title="East River Wind Event 2022-12-21 to 2022-12-27",
    xaxis4=dict(
        title="Time (UTC)"
    ),
    yaxis1=dict(
        title="Wind Speed (m/s)"
    ),
    yaxis2=dict(
        title="Sublimation\n(mm/hr)"
    ),
    yaxis3=dict(
        title="Pressure (hPa)"
    ),
    yaxis4=dict(
        title="Temperature (C)"
    ),
    legend_title="Measurement height\nabove ground (m)",
    hovermode="x unified",
    width= 800,
    height=800,
)

Create a SAIL wind rose for this event

In [94]:
# gather wspd_arith_mean and direction data from SOS at 10 meters on the UW tower
wind_variables = ['wspd_arith_mean','wdir_vec_mean']
met_wind_df = met_df[wind_variables].reset_index()
# filter to times on the 22nd of december
met_wind_df = met_wind_df.query("time.dt.day==22")

met_windrose_df = create_windrose_df(met_wind_df, wind_dir_var='wdir_vec_mean', wind_spd_var='wspd_arith_mean')

# create a wind rose plot using bar_polar
fig = px.bar_polar(met_windrose_df, 
                   r="frequency", 
                   theta="direction", 
                   color="speed", template="plotly_dark",
                   color_discrete_sequence= px.colors.sequential.Plasma_r,
                   width= 800,
                   height=500,
                   barnorm='percent',
                  )
# update the radial axis to show percentages
fig.update_layout(
    title="Wind Rose at 10 m from SAIL on 2022-12-22",
    font_size=16,
    polar_radialaxis=dict(
         ticksuffix='%',
         tickfont_size=14,
         showline=False,
         showticklabels=True,
         showgrid=True,
         angle=45,
         range=[0, 80]
      ),
)

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.


### Plot wind speed at other locations

In [19]:
import geopandas as gpd

In [20]:
# convert synoptic metadata into a geodataframe
synoptic_metadata_gdf = gpd.GeoDataFrame.from_dict(synoptic_metadata, orient='index')
# convert the coordinates to a point geometry
synoptic_metadata_gdf = synoptic_metadata_gdf.set_geometry(gpd.points_from_xy(synoptic_metadata_gdf.LONGITUDE, synoptic_metadata_gdf.LATITUDE))
# reset index and drop
synoptic_metadata_gdf = synoptic_metadata_gdf.reset_index(drop=True)
# set latitude, longitude and elevation to floats
synoptic_metadata_gdf.LATITUDE = synoptic_metadata_gdf.LATITUDE.astype(float)
synoptic_metadata_gdf.LONGITUDE = synoptic_metadata_gdf.LONGITUDE.astype(float)
synoptic_metadata_gdf['ELEVATION [ft]'] = synoptic_metadata_gdf['ELEVATION [ft]'].astype(float) 
# add CRS as 4326
synoptic_metadata_gdf.crs = 'epsg:4326'

In [150]:
# open snodgrass metadata
snodgrass_metadata = gpd.read_file('../../01_data/processed_data/east_river_stations/snodgrass_metadata_processed.geojson')
# convert elev_m to elev_ft
snodgrass_metadata['elev_ft'] = (snodgrass_metadata['elev_m']).astype(float)*3.28084
# convert lat and lon to float
snodgrass_metadata['lat'] = snodgrass_metadata['lat'].astype(float)
# rename index column to sta_name
snodgrass_metadata.rename(columns={'index':'sta_name'}, inplace=True)

,sta_name,lat,lon,elev_m,air temperature,relative humidity,avg. wind speed,max. wind speed,incoming shortwave radiation,reflected shortwave radiation,incoming longwave radiation,skin surface temperature,snow depth,accum precip,datetime,geometry,elev_ft
0,"Snodgrass Open AWS, mid-mountain",38.926572,-106.978929234,3132.24,deg C,%,m/s,m/s,W/m2,W/m2,W/m2,deg C,cm) or ground veg heigh,mm) -- open site onl,,POINT (-106.97893 38.92657),10276.378282
1,"Snodgrass Forest AWS, mid-mountain",38.929037,-106.977817542,3126.8,deg C,%,m/s,m/s,W/m2,W/m2,W/m2,deg C,cm) or ground veg heigh,None,,POINT (-106.97782 38.92904),10258.530512


In [21]:
# get token from .mapbox_token.json file in the home directory
with open('/home/dlhogan/.mapbox_token.json') as f:
    mapbox_token = json.load(f)['token']
f.close()

In [156]:
# normalize values between 0 and 1
synoptic_metadata_gdf['scale'] = 14 #synoptic_metadata_gdf['ELEVATION [ft]']/synoptic_metadata_gdf['ELEVATION [ft]'].max() * 20
fig = px.scatter_mapbox(synoptic_metadata_gdf,
                        lat="LATITUDE",
                        lon="LONGITUDE",
                        hover_name='STATION NAME',
                        hover_data=["ELEVATION [ft]","STATION"],
                        # color by elevation
                        color="ELEVATION [ft]",
                        color_continuous_scale=px.colors.sequential.Plasma_r,
                        size="scale",
                        # size_max=24,
                        zoom=10,
                        title="Synoptic Stations in Colorado",
                        )
# add the snodgrass stations
fig.add_trace(go.Scattermapbox(
    lat=snodgrass_metadata.lat,
    lon=snodgrass_metadata.lon,
    mode='markers',
    marker=go.scattermapbox.Marker(
        size=24,
        color=snodgrass_metadata.elev_ft,
        colorscale=px.colors.sequential.Plasma_r,
        showscale=False,
        opacity=0.7
    ),
    text=snodgrass_metadata.sta_name,
    showlegend=False,
    hoverinfo='text'
))
fig.update_layout(font_size=16,  title={'xanchor': 'center','yanchor': 'top', 'y':.98, 'x':0.5,}, 
        title_font_size = 24, mapbox_accesstoken=mapbox_token, mapbox_style = 'mapbox://styles/mapbox/outdoors-v12',
        width= 800, height=500, margin={"r":0,"t":40,"l":0,"b":0})

fig

In [350]:
filenames

['TAPC2.2023-12-31.csv',
 'CAIRW.2023-12-31.csv',
 'CASCP.2023-12-31.csv',
 'CACNM.2023-12-31.csv',
 'CO127.2023-12-31.csv',
 'DRBIL.2023-12-31.csv',
 'CACB1.2023-12-31.csv']

In [169]:
# Taylor Park
tapc2_df = load_synoptic_data(filepath, filenames[0], tidy=False)
tapc2_metadata = synoptic_metadata[tapc2_df.Station_ID.unique()[0]]
# Irwin Guides  plot
cairw_df = load_synoptic_data(filepath, filenames[1])
cairw_metadata = synoptic_metadata[cairw_df.Station_ID.unique()[0]]
# Irwin Guides ridge top
cascp_df = load_synoptic_data(filepath, filenames[2])
cascp_metadata = synoptic_metadata[cascp_df.Station_ID.unique()[0]]
# Cinnamon Mountain
cacnm_df = load_synoptic_data(filepath, filenames[3])
cacnm_metadata = synoptic_metadata[cacnm_df.Station_ID.unique()[0]]
# Red Lady Bowl
co127_df = load_synoptic_data(filepath, filenames[4])
co127_metadata = synoptic_metadata[co127_df.Station_ID.unique()[0]]
# Gothic billy DRI data
gothic_df = load_synoptic_data(filepath, filenames[5])
gothic_metadata = synoptic_metadata[gothic_df.Station_ID.unique()[0]]
# Crested Butte Mountain Resort
cbmr_df = load_synoptic_data(filepath, filenames[6])
cbmr_metadata = synoptic_metadata[cbmr_df.Station_ID.unique()[0]]
# Snodgrass open
snodgrass_opn_df = pd.read_csv('../../01_data/processed_data/east_river_stations/snodgrass_open_site_data_processed.csv', parse_dates=['datetime'], index_col=0)
# Snodgrass forest
snodgrass_for_df = pd.read_csv('../../01_data/processed_data/east_river_stations/snodgrass_forest_site_data_processed.csv', parse_dates=['datetime'], index_col=0)

Columns (2,3,4,5,6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.


In [170]:
# for each dataframe add a key to the metadata dictionary that calculates the temporal resolution of the data in minutes
for df, metadata in zip([tapc2_df, cairw_df, cascp_df, cacnm_df, co127_df, gothic_df, cbmr_df],
                        [tapc2_metadata, cairw_metadata, cascp_metadata, cacnm_metadata, co127_metadata, gothic_metadata, cbmr_metadata]):
    # if pressure and relative_humidity are in the columns, calculate specific humiditiy
    if ('pressure' in df.columns) & ('relative_humidity' in df.columns):
        df['specific_humidity'] = calc.specific_humidity_from_mixing_ratio(mixing_ratio=calc.mixing_ratio_from_relative_humidity(
                                                        relative_humidity=df['relative_humidity'].values * units.units.percent,
                                                        temperature=df['air_temp'].values * units.units.degC,
                                                        pressure=df['pressure'].values * units.units.pascal
                                                    )
                                                  )
    metadata['temporal_resolution'] = int(df.Date_Time.diff().dt.total_seconds().median()/60)

In [125]:
# print all the temporal resolutions
for k, v in synoptic_metadata.items():
    print(f"{k}: {v['temporal_resolution']} minutes")

TAPC2: 60 minutes
CAIRW: 60 minutes
CASCP: 60 minutes
CACNM: 15 minutes
CO127: 10 minutes
DRBIL: 10 minutes
CACB1: 15 minutes


In [171]:
# wind event start and end times
start_time = dt.datetime(2022,12,22,0,0,0, tzinfo=dt.timezone.utc)
end_time = dt.datetime(2022,12,23,0,0,0, tzinfo=dt.timezone.utc)
# make a color dictionary using the keys from the synoptic metadata dictionary
color_dict = dict(zip(synoptic_metadata.keys(), px.colors.qualitative.Plotly))
# create a figure with 3 facets: wind speed, wind direction, and temperature
fig = make_subplots(3,1, 
                    subplot_titles=("Wind Speed", "Specific Humidity", "Temperature"),
                    shared_xaxes=True, 
                    vertical_spacing=0.05)

for df in [tapc2_df, cairw_df, cascp_df, cacnm_df, co127_df, gothic_df, cbmr_df]:
    df_event_specific = df.query("Date_Time >= @start_time and Date_Time <= @end_time")
    # create the wind speed traces for the stations
    fig.add_trace(go.Scatter(
        x=df_event_specific['Date_Time'], 
        y=df_event_specific['wind_speed'],
        name=df.Station_ID.unique()[0],
        marker_color=color_dict[df_event_specific.Station_ID.unique()[0]]
    ),
    row=1, col=1
    )
    # if relative_humidity is in the columns then plot the relative humidity
    if ('relative_humidity' in df_event_specific.columns) and ('pressure' in df_event_specific.columns):
        fig.add_trace(go.Scatter(
            x=df_event_specific['Date_Time'], 
            y=df_event_specific['specific_humidity'],
            name=df.Station_ID.unique()[0],
            marker_color=color_dict[df_event_specific.Station_ID.unique()[0]]
        ),
        row=2, col=1
        )
    # create temperature traces for the stations
    fig.add_trace(go.Scatter(
        x=df_event_specific['Date_Time'], 
        y=df_event_specific['air_temp'],
        name=df_event_specific.Station_ID.unique()[0],
        marker_color=color_dict[df_event_specific.Station_ID.unique()[0]]
    ),
    row=3, col=1
    )
# for snowgrass data add the plots
for i,df in enumerate([snodgrass_opn_df, snodgrass_for_df]):
    # labels
    if i == 0:
        label = 'Snodgrass Open'
        color = 'black'
    else:
        label = 'Snodgrass Forest'
        color = 'forestgreen'
    df_event_specific = df.query("datetime >= @start_time and datetime <= @end_time")
    # create the wind speed traces for the stations
    fig.add_trace(go.Scatter(
        x=df_event_specific['datetime'], 
        y=df_event_specific['avg. wind speed'],
        name=label,
        marker_color=color
    ),
    row=1, col=1
    )
    # create temperature traces for the stations
    fig.add_trace(go.Scatter(
        x=df_event_specific['datetime'], 
        y=df_event_specific['air temperature'],
        name=label,
        marker_color=color
    ),
    row=3, col=1
    )
# add the met df wind speed trace for dec 22
fig.add_trace(go.Scatter(
    x=met_df_5min.query("time.dt.day==22")['time'],
    y=met_df_5min.query("time.dt.day==22")['wspd_arith_mean'],
    name='SAIL',
    marker_color='red'
),
row=1, col=1
)

# create the temperature traces for all stations
    
fig.update_layout(
    title="Wind Speed and Direction Synoptic Sites on 2022-12-22",
    xaxis3=dict(
        title="Time (UTC)"
    ),
    yaxis1=dict(
        title="Wind Speed (m/s)"
    ),
    yaxis2=dict(
        title="Specific Humidity (g/kg)"
    ),
    yaxis3=dict(
        title="Temperature (C)"
    ),
    legend_title="Station",
    hovermode="x unified",
    width= 800,
    height=800,
)
names = set()
fig.for_each_trace(
    lambda trace:
        trace.update(showlegend=False)
        if (trace.name in names) else names.add(trace.name))

In [175]:
# make subplots of wind rose for each site
for i,df in enumerate([tapc2_df, cairw_df, cascp_df, cacnm_df, co127_df, gothic_df, cbmr_df]):
    # get the station name from the unique station id 
    station_name = synoptic_metadata[df.Station_ID.unique()[0]]['STATION NAME']
    df_event_specific = df.query("Date_Time >= @start_time and Date_Time <= @end_time")
    # create wind rose dataframe for each station
    if ('wind_speed' in df_event_specific.columns) & ('wind_direction' in df_event_specific.columns):
        windrose_df = create_windrose_df(df_event_specific, wind_dir_var='wind_direction', wind_spd_var='wind_speed')
        # create a wind rose plot using bar_polar
        fig = px.bar_polar(windrose_df,
                            r="frequency", 
                            theta="direction", 
                            color="speed", template="plotly_dark",
                            color_discrete_sequence= px.colors.sequential.Plasma_r,
                            width= 400,
                            height=400,
                            barnorm='percent',
                              )
        # update the radial axis to show percentages
        fig.update_layout(
            title=f"Wind Rose at {station_name} on 2022-12-22",
            font_size=10,
            polar_radialaxis=dict(
                 ticksuffix='%',
                 tickfont_size=14,
                 showline=False,
                 showticklabels=True,
                 showgrid=True,
                 angle=45,
                 range=[0, 60]
              ),
        )
        fig.show()
    else:
        continue

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.


The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.


The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.


The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.


The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.


The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.


The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
